# Package import 

In [1]:
# Ne pas oublier d'executer dans le shell avant de lancer python :
# source /users/Enseignants/piwowarski/venv/amal/3.7/bin/activate
# N.B: commande pour lancer tensorboard: tensorboard --logdir=path/to/log-directory

import torch
from torch.autograd import Function
from torch.autograd import gradcheck
from datamaestro import prepare_dataset 
from torch.utils.tensorboard import SummaryWriter
import ipdb

# Functions 

In [3]:
class Context:
    """Very simplified context object"""
    def __init__(self):
        self._saved_tensors = ()
    def save_for_backward(self, *args):
        self._saved_tensors = args
    @property
    def saved_tensors(self):
        return self._saved_tensors


class linear(Function):
##Toute fonction a:
## une méthode forward pour calculer l'image de variables et paramètres donnés
## une méthode backward pour renvoyer son gradient par rapport à ses variables/paramètres
    @staticmethod
    #a static method can be called without referring to an object.     
    def forward(ctx,x,w,b):
        ctx.save_for_backward(x,w,b)
        return w*x+b

    @staticmethod
    def backward(ctx,grad_output):
        ## prend comme argument le gradient de l'étage d'au dessus
        x,w,b = ctx.saved_tensors
        dL_dx=grad_output*w
        dL_dw=grad_output*x
        dL_db=grad_output
        return dL_dx,dL_dw,dL_db
    
class MSE(Function):
    @staticmethod
    def forward(ctx,y,yhat):
        ctx.save_for_backward(y,yhat)
        return torch.mean((y-yhat)**2)
    @staticmethod
    def backward(ctx,grad_output=1):
        y,yhat=ctx.saved_tensors
        return 2*(y-yhat)*grad_output

x = torch.randn(10,5,requires_grad=True,dtype=torch.float64)
y=1

w = torch.randn(5,requires_grad=True,dtype=torch.float64)
b=torch.zeros(1,requires_grad=True, dtype=torch.float64)

## Pour utiliser la fonction 
f = linear()
ctx = Context()
output = f.forward(ctx,x,w,b)
f_grad = f.backward(ctx,y)

## Pour tester le gradient: 
## 1. Appliquer la méthode apply héritée de la classe Function pour créer une fonction
## 2. appliquer la fonction autograd_check à la fonction avec des paramètres
f_check = f.apply

print("Test of gradient for the linear function: ",torch.autograd.gradcheck(f_check,(x,w,b)))
#return True

##Test de la fonction MSE
yhat=output
mse=MSE()
loss=mse.forward(ctx,y,yhat)
print("the loss is: ",loss)
mse_grad=mse.backward(ctx)


Test of gradient for the linear function:  True
the loss is:  tensor(1.0404, dtype=torch.float64, grad_fn=<MeanBackward0>)


# Training 

## SGD descent

In [3]:
def SGD_descent(x,y,learning_rate,epochs=100):
    f=linear()
    mse=MSE()
    w = torch.randn(x.shape[1],requires_grad=True,dtype=torch.double)
    b = torch.zeros(1,dtype=torch.double)
    writer = SummaryWriter()
    print("SGD descent")
    for i in range(epochs):
        # Sample selection: one row of the sample selected randomly         
        index=torch.randint(0,len(x),size=(1,))
        x_i=torch.DoubleTensor(x[index,:])
        y_i=torch.DoubleTensor(y[index,:])
        #Context initialization
        ctx_f=Context()
        ctx_mse=Context()
        #Gradient calculation
        output=f.forward(ctx_f,x_i,w,b)
        loss=mse.forward(ctx_mse,output,y_i)
        mse_grad=mse.backward(ctx_mse)
        df_dx,df_dw,df_db=f.backward(ctx_f,mse_grad)
        #Parameters update
        w=w-learning_rate*df_dw
        b=b-learning_rate*df_db
        #Add loss to writer
        if i%10==0:
            print("Epoch %d"%i)
        writer.add_scalar('SGD_Loss', loss, i)
    return None

## Batch descent 

In [4]:
def Batch_descent(x,y,learning_rate,epochs=100):
    f=linear()
    mse=MSE()
    w = torch.randn(x.shape[1],requires_grad=True,dtype=torch.double)
    b = torch.zeros(1,requires_grad=True, dtype=torch.double)
    writer = SummaryWriter()
    print("Batch descent")
    for i in range(epochs):
        # Sample selection: the whole batch is considered
        for j in range(len(x)):
            x_j=torch.DoubleTensor(x[j,:])
            y_j=torch.DoubleTensor(y[j,:])
            #Context initialization
            ctx_f=Context()
            ctx_mse=Context()
            #Gradient calculation
            output=f.forward(ctx_f,x_j,w,b)
            loss=mse.forward(ctx_mse,output,y_j)
            mse_grad=mse.backward(ctx_mse)
            df_dx,df_dw,df_db=f.backward(ctx_f,mse_grad)
        #Parameters update
        w=w-learning_rate*df_dw
        b=b-learning_rate*df_db
        #Add loss to writer
        if i%10==0:
            print("Epoch %d"%i)
        writer.add_scalar('Batch_Loss', loss, i)
    return None

## Mini-batch 

In [5]:
def MiniBatch_descent(x,y,learning_rate,epochs=100,batch_size=1):
    f=linear()
    mse=MSE()
    w = torch.randn(1,x.shape[1],requires_grad=True,dtype=torch.double)
    b = torch.zeros(1,dtype=torch.double)
    writer = SummaryWriter()
    print("MiniBatch descent")
    for i in range(epochs):
        # Sample selection: batch_size rows of the sample selected randomly         
        index=torch.randint(0,len(x),size=(batch_size,))
        for j in index:
            x_j=torch.DoubleTensor(x[j,:])
            y_j=torch.DoubleTensor(y[j,:])
            #Context initialization
            ctx_f=Context()
            ctx_mse=Context()
            #Gradient calculation
            output=f.forward(ctx_f,x_j,w,b)
            loss=mse.forward(ctx_mse,output,y_j)
            mse_grad=mse.backward(ctx_mse)
            df_dx,df_dw,df_db=f.backward(ctx_f,mse_grad)
        #Parameters update
        w=w-learning_rate*df_dw
        b=b-learning_rate*df_db
        #Add loss to writer 
        if i%10==0:
            print("Epoch %d"%i)
        writer.add_scalar('Minibatch_Loss', loss, i)
    return None

## Tests 

In [6]:
n=10
x=torch.randn(n,5,requires_grad=True,dtype=torch.double)
y=torch.randn(n,1,dtype=torch.double)

epochs=1000
learning_rate=0.001
batch_size=int(n/2)

# SGD_descent(x,y,learning_rate,epochs)
# Batch_descent(x,y,learning_rate,epochs)
# MiniBatch_descent(x,y,learning_rate,epochs,batch_size)


# Boston dataset 

In [7]:
## Téléchargement du dataset Boston
ds=prepare_dataset("edu.uci.boston")
fields, data =ds.files.data() 

print("fields:",fields)
print("dataset",data)

x=torch.DoubleTensor(data[:,:-1])
y=torch.DoubleTensor(data[:,-1]).reshape(len(x),1)

def standardize(z):
    mu = z.mean(dim=0, keepdim=True)
    std = z.std(dim=0, keepdim=True)
    return (z - mu) / std

x=standardize(x)
y=standardize(y)

fields: ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
dataset [[6.3200e-03 1.8000e+01 2.3100e+00 ... 3.9690e+02 4.9800e+00 2.4000e+01]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 3.9690e+02 9.1400e+00 2.1600e+01]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 3.9283e+02 4.0300e+00 3.4700e+01]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 3.9690e+02 5.6400e+00 2.3900e+01]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 3.9345e+02 6.4800e+00 2.2000e+01]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 3.9690e+02 7.8800e+00 1.1900e+01]]


In [10]:
epochs=1000
learning_rate=0.1
batch_size=int(n/10)

SGD_descent(x,y,learning_rate,epochs)
Batch_descent(x,y,learning_rate,epochs)
MiniBatch_descent(x,y,learning_rate,epochs,batch_size)

SGD descent
Epoch 0
Epoch 10
Epoch 20
Epoch 30
Epoch 40
Epoch 50
Epoch 60
Epoch 70
Epoch 80
Epoch 90
Epoch 100
Epoch 110
Epoch 120
Epoch 130
Epoch 140
Epoch 150
Epoch 160
Epoch 170
Epoch 180
Epoch 190
Epoch 200
Epoch 210
Epoch 220
Epoch 230
Epoch 240
Epoch 250
Epoch 260
Epoch 270
Epoch 280
Epoch 290
Epoch 300
Epoch 310
Epoch 320
Epoch 330
Epoch 340
Epoch 350
Epoch 360
Epoch 370
Epoch 380
Epoch 390
Epoch 400
Epoch 410
Epoch 420
Epoch 430
Epoch 440
Epoch 450
Epoch 460
Epoch 470
Epoch 480
Epoch 490
Epoch 500
Epoch 510
Epoch 520
Epoch 530
Epoch 540
Epoch 550
Epoch 560
Epoch 570
Epoch 580
Epoch 590
Epoch 600
Epoch 610
Epoch 620
Epoch 630
Epoch 640
Epoch 650
Epoch 660
Epoch 670
Epoch 680
Epoch 690
Epoch 700
Epoch 710
Epoch 720
Epoch 730
Epoch 740
Epoch 750
Epoch 760
Epoch 770
Epoch 780
Epoch 790
Epoch 800
Epoch 810
Epoch 820
Epoch 830
Epoch 840
Epoch 850
Epoch 860
Epoch 870
Epoch 880
Epoch 890
Epoch 900
Epoch 910
Epoch 920
Epoch 930
Epoch 940
Epoch 950
Epoch 960
Epoch 970
Epoch 980
Epoch 990